Module 4 Essentials Project

Michael O'Donnell 02/01/2024

--------------------------------

For this assignment, I will be using Selenium and Beautiful soup to scrape Gab for posts that disucuss the keyword 'Taylor Swift' since Gab is used by right-wingers and right-wingers are targeting Taylor with conspiracy theorie. Here is an article discussing this:

https://www.msn.com/en-us/sports/nfl/right-wing-media-figures-target-taylor-swift-with-absurd-conspiracy-theory-ahead-of-the-super-bowl/ar-BB1htK8Q

Before this conspiracy becomes irrelevant and is replaced by another conspiracy, probably against her sadly, I want to scrape and archive what the users of the platform are saying about her and try to gain some insight into the reasonings behind her being involved in a conspiracy theory.

---------------------------

I used many resources for this assignment along with some handy dandy help from ChatGPT when I was stuck in the mud.

Below are some resources that I used when coding this assignment:

https://stackoverflow.com/questions/43835404/importing-keys-from-selenium-webdriver-common-keys

https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

https://stackoverflow.com/questions/7271385/how-do-i-combine-two-lists-into-a-dictionary-in-python

https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/#

https://stackoverflow.com/questions/53141240/pandas-how-to-swap-or-reorder-columns

https://www.easytweaks.com/add-column-current-date-timestamp-pandas/

https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/#

In [333]:
#Import Statements
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime as dt
import time

In [334]:
#Launches Chrome that I can manipulate
driver=webdriver.Chrome()

In [335]:
#Goes to Gab
driver.get('https://gab.com/')

In [336]:
#This accesses the search and searches for Taylor Swift
time.sleep(5)
driver.find_element(By.ID,'nav-search').send_keys('Taylor Swift' + Keys.ENTER)

In [337]:
#This scrolls twice to load in the first set of posts. For some reason Gab only pulls a preview for the first intitial load.
time.sleep(5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#This then adds the source code to a variable to store it
pagesource += driver.page_source

In [338]:
#This for loop takes the above logic and then scrolls to the bottom of the page 5 times while adding additional code to the variable
for i in range(0,4):
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    pagesource += driver.page_source

I noticed here that around 5 or so additional scrolls from the time I load in goes to about 2 hours worth of posts. So if I wanted this to run continually, I think I would run this every couple hours and have a way to make sure at the bottom that duplicated posts are not being added to the CSV.

In [339]:
#Since we have the HTML we want to look over stored in a variable, we no longer need the website up and running. 
time.sleep(10)
driver.quit()

In [340]:
#This puts the HTML we gathered from the Selenium variable into a beautiful soup variable and parses it with HTML
bs = BeautifulSoup(pagesource,'html.parser')

In [341]:
#This finds the overarching div that contains the profile name and the body of text in the post
meta_data_posts = bs.find_all('div', class_='_81_1w')

In [342]:
#This sets two list variables that are going to hold the names and texts once they are delimitted
post_texts = []
author_names = []

In [343]:
#This chunk of code loops through all of the beautiful soup variable/selenium variable with the html code grabbed from Gab
#that then stores the author and text posts in variables. Those variables are then stored in corresponding lists if they match
#each other from the metadata, so the posts and authors don't get mixed out of order
for post in meta_data_posts:
    post_text = post.find('div',class_='_1FwZr _81_1w')
    author = post.find('div', class_='_UuSG Naf1t _3dGg1 ALevz')

    if post_text and author:
        post_texts.append(post_text.get_text(strip=True))
        author_names.append(author.get_text(strip=True))

In [344]:
#Checks to make sure that the columns will be the same length
print(len(post_texts))
print(len(author_names))

1330
1330


In [345]:
#This creates a new dataframe with the lists of names and posts
df = pd.DataFrame({
    'Author Name':author_names,
    'Post Text':post_texts           
})

In [346]:
#this drops duplicate rows
df = df.drop_duplicates()

In [347]:
#This drops rows with empty cells
df = df.dropna()

In [348]:
#Checks the max number of splits that will be occuring
max_splits = df['Author Name'].str.count('@').max()

In [349]:
#Splits the author with the max number of @s so we can see the account name and account handle seperately
split_cols = df['Author Name'].str.split('@', n=max_splits, expand=True)

In [350]:
#Makes a new dataframe with the dataframe of split columns
new_df = pd.concat([df, split_cols], axis=1)

In [351]:
#Since the posts don't include really include a time when posted, I added a column to show when these posts were grabbed
#by the crawler. And in theory, since this would be running every 2 hours or so, this would be sort of accurate. It would
#at least be pretty accurate on the date, but not the time. 
new_df['Post Grabbed Time'] = pd.Timestamp('now')

In [352]:
#Drops a column we do not need
new_df = new_df.drop(2, axis=1)

In [353]:
#Finds the number of verified accounts
max_splits2 = new_df[0].str.count('Verified Account').max()

In [354]:
#Splits the accounts where Verified Account is found in the author name
split_cols2 = new_df[0].str.split('Verified Account', n=max_splits2, expand=True)

In [355]:
#Renames dataframe columns of columns that were split to make more sense
split_cols2.rename(columns = {0:'Account Name'}, inplace = True)
split_cols2.rename(columns = {1:'Verified Account'}, inplace = True)
new_df.rename(columns = {0:'Full Account Name'}, inplace = True)
new_df.rename(columns = {1:'Account Handle'}, inplace = True)

In [356]:
#Combines the dataframes to a new dataframe
df2 = pd.concat([new_df, split_cols2], axis=1)

In [357]:
#Sets the accounts that are verified with Verified 
df2['Verified Account'] = df2['Verified Account'].replace('','Verified')

In [358]:
#Sets the accounts that are not verified with Not Verified
df2['Verified Account'] = df2['Verified Account'].replace(to_replace=[None],value='Not Verified')

In [359]:
#drops the Full Account Name column since we have that info split up
df2 = df2.drop('Full Account Name', axis=1)

In [360]:
#Sets a new dataframe that reogranizes the df2 columns
final_df = df2[['Account Name', 'Account Handle', 'Verified Account', 'Post Text', 'Post Grabbed Time', 'Author Name']]

In [361]:
#prints the final dataframe to make sure everything looks good
final_df

,Account Name,Account Handle,Verified Account,Post Text,Post Grabbed Time,Author Name
0,Luis Rosales,ese_cholito,Not Verified,"Bad Bunny, Taylor Swift and MORE are taken off...",2024-02-02 17:04:06.252082,Luis Rosales@ese_cholito
2,Free_Talk,Free_Talk,Not Verified,"This swift is a nobody, why on earth do people...",2024-02-02 17:04:06.252082,Free_Talk@Free_Talk
4,Barbara Campbell,dogmama53,Not Verified,Proof Taylor Swift is a psyopMarilyn@Jaizemthi...,2024-02-02 17:04:06.252082,Barbara Campbell@dogmama53
6,Erich Brink,Indignatie,Not Verified,https://indignatie.nl/trump-bondgenoten-belove...,2024-02-02 17:04:06.252082,Erich Brink@Indignatie
10,klsparrow,klsparrow,Not Verified,Swift has not yet endorsed anyone in the 2024 ...,2024-02-02 17:04:06.252082,klsparrow@klsparrow
...,...,...,...,...,...,...
1070,Augusto’s Helicopter Tours,LaResistance,Not Verified,Taylor Swift is not a psyop ... I beg to disag...,2024-02-02 17:04:06.252082,Augusto’s Helicopter Tours@LaResistance
1072,Veritas Lux Mea,Veritas_Lux_Mea,Not Verified,https://nationalfile.com/mr-pfizer-travis-kelc...,2024-02-02 17:04:06.252082,Veritas Lux Mea@Veritas_Lux_Mea
1074,John Bernhard,PatriotJohnB52,Not Verified,https://nationalfile.com/mr-pfizer-travis-kelc...,2024-02-02 17:04:06.252082,John Bernhard@PatriotJohnB52
1076,Gabriel’s Horn,Gabriels_Horn,Not Verified,https://www.breitbart.com/politics/2024/02/01/...,2024-02-02 17:04:06.252082,Gabriel’s Horn@Gabriels_Horn


In [362]:
#Exports the final version of the dataframe to CSV
final_df.to_csv('GabPosts.csv')

Post Assignment Thoughts:

I initially wanted to crawl Truth Social, but I had trouble logging in. Gab was a good back up to crawl since it was easy to find the elements I needed and I did not need to make an account or anything. It was quite easy to access. 

An issue that isn't considered in this code is that many posts contained pictures which I did not get. This seems to be the primary way of sharing information on the site. If I was going to revisit this later on, I would want to add the pictures as well. 

As I mentioned above, I would ideally want this to run every 2 hours or so and then update the CSV with rows that would not be duplicates which is not included in this project. Although I consider this project to accomplish what is being asked for the assignment, there are still pieces to it that could be added. 